In [2]:
import pandas as pd

In [24]:
df_ss = pd.read_csv('sample_submission.csv')
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [25]:
print(df_ss['target'].unique())
df_train.info()

[0]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24712 entries, 0 to 24711
Data columns (total 22 columns):
_id               24712 non-null object
age               24712 non-null int64
job               24712 non-null object
marital           24712 non-null object
education         24712 non-null object
default           24712 non-null object
housing           24712 non-null object
loan              24712 non-null object
contact           24712 non-null object
month             24712 non-null object
day_of_week       24712 non-null object
duration          24712 non-null int64
campaign          24712 non-null int64
pdays             24712 non-null int64
previous          24712 non-null int64
poutcome          24712 non-null object
emp.var.rate      24712 non-null float64
cons.price.idx    24712 non-null float64
cons.conf.idx     24712 non-null float64
euribor3m         24712 non-null float64
nr.employed       24712 non-null float64
target            24712 non-null int64
dtypes: 

In [5]:
# Проверяем, совпадает ли порядок id адресов клиентов в тренировочной выборке и файле 'sample_submission'

check = df_train['_id'] == df_ss['_id']
print(check.unique())

[ True]


In [44]:
# создаем функцию для предобработке данных

def month_to_digit(m):
    months_dict = {'mar':0, 'apr':1, 'may':2, 'jun':3, 'jul':4, 'aug':5, 'sep':6, 'oct':7, 'nov':8, 'dec':9}
    return months_dict[m]

def day_to_digit(d):
    days_dict = {'mon':0, 'tue':1, 'wed':2, 'thu':3, 'fri':4}
    return days_dict[d]

def prepare_data(df):
    
    df = df.drop(['_id'], axis=1) # Удаляем индивидуальный ID клиента

    df = df.drop(['target'], axis=1) # Удаляем индивидуальный ID клиента
    
    df['month'] = df['month'].map(lambda x: month_to_digit(x)) # Конвертируем названия месяцев в числа
    
    df['day_of_week'] = df['day_of_week'].map(lambda x: day_to_digit(x)) # Конвертируем дни недели в числа
    
    category_columns = [
        'job',
        'marital',
        'education',
        'default',
        'housing',
        'loan',
        'contact',
        'poutcome'
    ]
    
    df = pd.get_dummies(df, prefix=category_columns)
    
    
    
    return df
    
    

In [45]:
X = prepare_data(df_train)
y = df_train['target'].values

In [46]:
# Разделяем датасет на тренировочную и тестовую выборки
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Делаем Scaling
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [47]:
# Импортируем модели классификации

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
knn.fit(X_train, y_train)
knn_predict = knn.predict(X_test)
knn_predict_proba = knn.predict_proba(X_test)

dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
dtc_predict = dtc.predict(X_test)
dtc_predict_proba = dtc.predict_proba(X_test)

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr_predict = lr.predict(X_test)
lr_predict_proba = lr.predict_proba(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

## Your Code Here
print('KNeighborsClassifier metrics')
print('accuracy', accuracy_score(y_test, knn_predict))
print('precision', precision_score(y_test, knn_predict))
print('recall', recall_score(y_test, knn_predict))
print('')

print('DecisionTreeClassifier metrics')
print('accuracy', accuracy_score(y_test, dtc_predict))
print('precision', precision_score(y_test, dtc_predict))
print('recall', recall_score(y_test, dtc_predict))
print('')

print('LogisticRegression metrics')
print('accuracy', accuracy_score(y_test, lr_predict))
print('precision', precision_score(y_test, lr_predict))
print('recall', recall_score(y_test, lr_predict))
print('')

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score, roc_curve

knn_fpr, knn_tpr, knn_thresholds = roc_curve(y_test, knn_predict_proba[:,1])
dtc_fpr, dtc_tpr, dtc_thresholds = roc_curve(y_test, dtc_predict_proba[:,1])
lr_fpr, lr_tpr, lr_thresholds = roc_curve(y_test, lr_predict_proba[:,1])
plt.figure(figsize=(10, 10))

plt.plot(knn_fpr, knn_tpr, label='knn')
#plt.legend('dtc')

plt.plot(dtc_fpr, dtc_tpr, label='dtc')
#plt.legend('rfc')

plt.plot(lr_fpr, lr_tpr, label='lr')
#plt.legend('lr')

plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.plot([0, 1], [0, 1])
plt.ylabel('tpr')
plt.xlabel('fpr')
plt.grid(True)
plt.title('ROC curve')
plt.xlim((-0.01, 1.01))
plt.ylim((-0.01, 1.01))

plt.show(y_test.shape)
plt.show(dtc_predict_proba.shape)

knn_roc_auc_score = roc_auc_score(y_test, knn_predict)
print('KNeighborsClassifier roc_auc: {}'.format(knn_roc_auc_score))

dtc_roc_auc_score = roc_auc_score(y_test, dtc_predict)
print('DecisionTreeClassifier roc_auc: {}'.format(dtc_roc_auc_score))

lr_roc_auc_score = roc_auc_score(y_test, lr_predict)
print('LogisticRegression roc_auc: {}'.format(lr_roc_auc_score))
print('')